In [1]:
!pip install d2l==1.0.0-beta0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.6/141.6 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.6/155.6 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.0 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616821 sha256=7a208671999160d7432fe453c1c5a97e1bd1245f2afb7d27eb4962848705fcf6
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successfully built gym
  Attempting uninstal

Bây giờ chúng ta đã hiểu cách các lớp tích chập hoạt động trên lý thuyết, chúng ta đã sẵn sàng để xem chúng hoạt động như thế nào trong thực tế. Dựa trên động lực của chúng ta về các mạng thần kinh tích chập như là kiến ​​trúc hiệu quả để khám phá cấu trúc trong dữ liệu hình ảnh, chúng tôi sử dụng hình ảnh làm ví dụ đang chạy của mình.

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# 7.2.1. Toán tử tương quan chéo

Như ta đã biết, tầng tích chập là cái tên có phần không chính xác, vì phép toán mà chúng biểu diễn là phép tương quan chéo (cross correlation). Trong một tầng tích chập, một mảng đầu vào và một mảng hạt nhân tương quan được kết hợp để tạo ra mảng đầu ra bằng phép toán tương quan chéo. Hãy tạm thời bỏ qua chiều kênh và xem phép toán này hoạt động như thế nào với dữ liệu và biểu diễn ẩn hai chiều. Trong hình dưới, đầu vào là một mảng hai chiều với chiều dài 3 và chiều rộng 3. Ta kí hiệu kích thước của mảng là $3×3$ hoặc $(3, 3)$. Chiều dài và chiều rộng của hạt nhân đều là 2. Chú ý rằng trong cộng đồng nghiên cứu học sâu, mảng này còn có thể được gọi là hạt nhân tích chập, bộ lọc hay đơn thuần là trọng số của tầng. Kích thước của cửa sổ hạt nhân là chiều dài và chiều rộng của hạt nhân (ở đây là $2×2$).

![Two-dimensional cross-correlation operation. The shaded portions are the first output element as well as the input and kernel tensor elements used for the output computation: $0\times0+1\times1+3\times2+4\times3=19$.](http://d2l.ai/_images/correlation.svg)

Phép tương quan chéo hai chiều. Các phần được tô màu là phần tử đầu tiên của đầu ra cùng với các phần tử của mảng đầu vào và mảng hạt nhân được sử dụng trong phép toán:
$$
0\times0+1\times1+3\times2+4\times3=19
$$

Trong phép tương quan chéo hai chiều, ta bắt đầu với cửa sổ tích chập đặt tại vị trí góc trên bên trái của mảng đầu vào và di chuyển cửa sổ này từ trái sang phải và từ trên xuống dưới. Khi cửa sổ tích chập được đẩy tới một vị trí nhất định, mảng con đầu vào nằm trong cửa sổ đó và mảng hạt nhân được nhân theo từng phần tử, rồi sau đó ta lấy tổng các phần tử trong mảng kết quả để có được một giá trị số vô hướng duy nhất. Giá trị này được ghi vào mảng đầu ra tại vị trí tương ứng. Ở đây, mảng đầu ra có chiều dài 2 và chiều rộng 2, với bốn phần tử được tính bằng phép tương quan chéo hai chiều:

$$
\begin{split}0\times0+1\times1+3\times2+4\times3=19,\\
1\times0+2\times1+4\times2+5\times3=25,\\
3\times0+4\times1+6\times2+7\times3=37,\\
4\times0+5\times1+7\times2+8\times3=43.\end{split}
$$

Lưu ý rằng dọc theo mỗi trục, kích thước đầu ra nhỏ hơn một chút so với kích thước đầu vào. Vì hạt nhân có chiều rộng và chiều cao lớn hơn một, nên chúng tôi chỉ có thể tính toán đúng mối tương quan chéo cho các vị trí mà hạt nhân hoàn toàn khớp với hình ảnh, kích thước đầu ra được cung cấp bởi kích thước đầu vào $n_h \times n_w$ trừ đi kích thước của hạt tích chập $k_h \times k_w$ thông qua 
$$
(n_h-k_h+1) \times (n_w-k_w+1).
$$
Đây là trường hợp vì chúng ta cần đủ không gian để “chuyển” nhân tích chập qua hình ảnh. Sau này chúng ta sẽ xem cách giữ nguyên kích thước bằng cách đệm hình ảnh với các số 0 xung quanh ranh giới của nó để có đủ không gian để dịch chuyển nhân. Tiếp theo, chúng tôi triển khai quy trình này trong hàm corr2d, hàm chấp nhận một tenxơ đầu vào X và một tenxơ kernel K và trả về một tenxơ đầu ra Y.

In [16]:
def corr2d(X, K):
  h, w = K.shape
  Y = torch.zeros((X.shape[0]-h+1, X.shape[1]-w+1))
  for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
      Y[i][j] = (X[i:i+h,j:j+w]*K).sum()
  return Y

Chúng ta có thể xây dựng tenxơ đầu vào X và tenxơ hạt nhân K từ hình trên để xác thực đầu ra của việc triển khai hoạt động tương quan chéo hai chiều ở trên.

In [17]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

(tensor([[19., 25.],
         [37., 43.]]), tensor(1.))

# 7.2.2. Lớp tích chập

Một lớp tích chập tương quan chéo giữa đầu vào và nhân và thêm độ lệch vô hướng để tạo ra đầu ra. Hai tham số của một lớp tích chập là hạt nhân và độ lệch vô hướng. Khi đào tạo các mô hình dựa trên các lớp tích chập, chúng ta thường khởi tạo các hạt nhân một cách ngẫu nhiên, giống như chúng ta làm với một lớp được kết nối đầy đủ.

Bây giờ chúng ta đã sẵn sàng để triển khai lớp tích chập hai chiều dựa trên hàm corr2d được xác định ở trên. Trong `__init__` phương thức khởi tạo, chúng ta khai báo weight và bias là hai tham số của mô hình. Phương pháp lan truyền thuận gọi hàm corr2d và thêm độ lệch.


In [18]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

TRONG $h \times w$ tích chập hoặc một $h \times w$ nhân tích chập, chiều cao và chiều rộng của nhân tích chập là $h$
Và $w$, tương ứng. Chúng ta cũng đề cập đến một lớp tích chập với một $h \times w$ nhân tích chập đơn giản như một $h \times w$ lớp tích chập.

# 7.2.3. Phát hiện Biên của Vật thể trong Ảnh

Hãy dành một chút thời gian để phân tích một ứng dụng đơn giản của lớp tích chập: phát hiện cạnh của một đối tượng trong ảnh bằng cách tìm vị trí thay đổi pixel. Đầu tiên, chúng tôi xây dựng một “hình ảnh” của $6\times 8$ điểm ảnh. Bốn cột ở giữa có màu đen (0) và phần còn lại có màu trắng (1).

In [19]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

Tiếp theo, chúng ta xây dựng một hạt nhân Kcó chiều cao là 1 và chiều rộng là 2. Khi chúng ta thực hiện thao tác tương quan chéo với đầu vào, nếu các phần tử liền kề theo chiều ngang giống nhau thì đầu ra là 0. Ngược lại, đầu ra là không số không. Lưu ý rằng nhân này là trường hợp đặc biệt của toán tử sai phân hữu hạn. Tại vị trí $(i,j)$ nó tính toán $x_{i,j} - x_{(i+1),j}$, nghĩa là, nó tính toán sự khác biệt giữa các giá trị của các pixel liền kề theo chiều ngang. Đây là một xấp xỉ rời rạc của đạo hàm đầu tiên theo hướng nằm ngang. Rốt cuộc, đối với một chức năng $f(i,j)$
phái sinh của nó $-\partial_i f(i,j) = \lim_{\epsilon \to 0} \frac{f(i,j) - f(i+\epsilon,j)}{\epsilon}$. Hãy xem làm thế nào điều này hoạt động trong thực tế.

In [20]:
K = torch.tensor([[1.0, -1.0]])

Chúng ta đã sẵn sàng thực hiện thao tác tương quan chéo với các đối số X(đầu vào của chúng ta) và K(nhân của chúng ta). Như ta có thể thấy, chúng ta phát hiện 1 cho cạnh từ trắng sang đen và -1 cho cạnh từ đen sang trắng. Tất cả các đầu ra khác có giá trị 0.

In [23]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

Bây giờ chúng ta có thể áp dụng hạt nhân cho hình ảnh được chuyển vị. Đúng như dự đoán, nó biến mất. Nhân K chỉ phát hiện các cạnh dọc.

In [24]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

# 7.2.4. Học một Bộ lọc

Việc thiết kế bộ phát hiện biên bằng sai phân hữu hạn [1, -1] thì khá gọn gàng nếu ta biết chính xác đây là những gì cần làm. Tuy nhiên, khi xét tới các bộ lọc lớn hơn và các tầng tích chập liên tiếp, việc chỉ định chính xác mỗi bộ lọc cần làm gì một cách thủ công là bất khả thi.

Bây giờ ta hãy xem liệu có thể học một bộ lọc có khả năng tạo ra Y từX chỉ từ các cặp (đầu vào, đầu ra) hay không. Đầu tiên chúng ta xây dựng một tầng tích chập và khởi tạo một mảng ngẫu nhiên làm bộ lọc. Tiếp theo, trong mỗi lần lặp, ta sẽ sử dụng bình phương sai số để so sánh Y và đầu ra của tầng tích chập, sau đó tính toán gradient để cập nhật trọng số. Để đơn giản, trong tầng tích chập này, ta sẽ bỏ qua hệ số điều chỉnh.


In [28]:
# Khởi tạo lớp tích chập
conv2d = nn.LazyConv2d(1, kernel_size=(1,2),bias = False)
# Khởi tạo mẫu
X = X.reshape((1,1,6,8))
Y = Y.reshape((1,1,6,7))
lr = 0.03

for i in range(10):
  Y_hat = conv2d(X)
  l = (Y_hat - Y)**2
  conv2d.zero_grad()
  l.sum().backward()
  #Cập nhật weight
  conv2d.weight.data[:] -= lr*conv2d.weight.grad
  if (i+1) & 2 == 0:
    print(f"epoch {i+1}, loss = {l.sum():.3f}")

epoch 1, loss = 21.773
epoch 4, loss = 2.011
epoch 5, loss = 0.984
epoch 8, loss = 0.154
epoch 9, loss = 0.090


Lưu ý rằng lỗi đã giảm xuống một giá trị nhỏ sau 10 lần lặp lại. Bây giờ chúng ta sẽ xem xét tensor hạt nhân mà chúng ta đã học.

In [30]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9628, -1.0081]])

Thật vậy, tenxơ hạt nhân đã học rất gần với tenxơ hạt nhân mà Kchúng ta đã định nghĩa trước đó.

# 7.2.5. Tương quan Chéo và Tích chập

Hãy nhớ lại kiến thức của phần trước về mối liên hệ giữa phép tương quan chéo và tích chập. Trong hình trên, ta dễ dàng nhận thấy điều này. Đơn giản chỉ cần lật bộ lọc từ góc dưới cùng bên trái lên góc trên cùng bên phải. Trong trường hợp này, chỉ số trong phép lấy tổng được đảo ngược, nhưng ta vẫn thu được kết quả tương tự. Để thống nhất với các thuật ngữ tiêu chuẩn trong tài liệu học sâu, ta sẽ tiếp tục đề cập đến phép tương quan chéo như là phép tích chập, mặc dù đúng ra chúng hơi khác nhau một chút.